In [1]:
## Description

In [2]:
## Gathering

In [6]:
### Twitter archive enhanced

In [10]:
### Tweet image predictions

In [11]:
### Tweet attributes

In [12]:
##  Assessing

In [13]:
### Visual assesment 

In [14]:
### Programmatic assesment

In [ ]:
### Documenting Issues

In [ ]:
#### Quality issues

In [ ]:
#### Tidiness issues

In [4]:
## Cleaning

In [ ]:
### Quality issues

In [ ]:
### Tidiness issues 

In [ ]:
## Storing

In [ ]:
### Twitter archive master to a csv file

In [ ]:
### Twitter arhive master to a sqlite database

In [5]:
## Analysis and visualization

In [ ]:
### Most tweeted dog stage

In [ ]:
### Average popularity rating of WeRateDogs

In [ ]:
### Popularity trend of WeRateDogs